# Task
Write a simple Python script/program that takes two date arguments and retrieves the list of near-Earth space objects approaching Earth in that time interval. Output the list of the objects, sorted by their closest approach distance, in an aligned tabular format, the object name, size estimate, time and distance of the closest encounter.
The data should come from the API of NeoWs service at https://api.nasa.gov/ (free registration required).

In [183]:
import requests
import pandas as pd

# Question: Will the fall of a space object disrupt my birthday party in 2022?
Let's get the data and store it as json file.

In [184]:
response = requests.get("https://api.nasa.gov/neo/rest/v1/feed?start_date=2022-05-12&end_date=2022-05-19&api_key=qVNuxuWB0l7e3SRfNz8USva2h3IOUyj7HaxutEc8")

json_data = response.json()
df = pd.DataFrame(json_data)
df.to_json('nasa_neows.json')

Read data and drop na.

In [185]:
planets_df = pd.read_json("nasa_neows.json")
planets_df = planets_df[planets_df['near_earth_objects'].notna()]
planets_df

,links,element_count,near_earth_objects
2022-05-18,None,66,[{'links': {'self': 'http://www.neowsapp.com/r...
2022-05-19,None,66,[{'links': {'self': 'http://www.neowsapp.com/r...
2022-05-16,None,66,[{'links': {'self': 'http://www.neowsapp.com/r...
2022-05-17,None,66,[{'links': {'self': 'http://www.neowsapp.com/r...
2022-05-14,None,66,[{'links': {'self': 'http://www.neowsapp.com/r...
2022-05-15,None,66,[{'links': {'self': 'http://www.neowsapp.com/r...
2022-05-12,None,66,[{'links': {'self': 'http://www.neowsapp.com/r...
2022-05-13,None,66,[{'links': {'self': 'http://www.neowsapp.com/r...


Let's write a function that will search "near_earth_objects" column for all the information we need: Object name, size, time and distance of the closest encounter.

In [186]:
def dates(df, column, row):
    
# define the date we will work with
    date = df[column][row]
    date = pd.json_normalize(date, max_level=2)
    
# drop unnecessary columns
    date = date[['name','close_approach_data','estimated_diameter.meters.estimated_diameter_min',
                 'estimated_diameter.meters.estimated_diameter_max']]

# we need the size of an object - let's calculate the averge from min and max values
    date["size_estimate_m"] = (date['estimated_diameter.meters.estimated_diameter_min'] +
                                date['estimated_diameter.meters.estimated_diameter_max'])/2
    
# get time and distance of the closest encounter from "close_approach_data" column
    times = []
    closest_approach_distance = []
    for idx, object in date.iterrows():
        close_approach_data = date["close_approach_data"][idx]
        close_approach_data = pd.json_normalize(close_approach_data, max_level=2)
        object_closest_approach_distance = close_approach_data["miss_distance.kilometers"].values
        object_closest_approach_distance = object_closest_approach_distance[0]
        closest_approach_distance.append(object_closest_approach_distance)
        full_date = close_approach_data["close_approach_date_full"].values
        full_date = full_date[0]
        times.append(full_date)

# add times and closest approach distances to dataframe
    date["time"] = times
    date["closest_approach_distance_m"] = closest_approach_distance
    
# drop columns we don't need
    date = date.drop(columns=['estimated_diameter.meters.estimated_diameter_min',
                              'estimated_diameter.meters.estimated_diameter_max',
                              'close_approach_data'])
# return dataframe
    return date

Let's create empty dataframe to store our data into.

In [187]:
final_planets_df = pd.DataFrame(columns=["name", "size_estimate_m", "time", "closest_approach_distance_m"])

Let's check every row of original dataframe to get clean data.

In [188]:
for idx, date in planets_df.iterrows():
    df = dates(planets_df, 'near_earth_objects', idx)
    final_planets_df = pd.concat([final_planets_df, df], ignore_index=True)

Let's check how the final dataframe looks like.

In [189]:
final_planets_df

,name,size_estimate_m,time,closest_approach_distance_m
0,(2010 UY7),8.581093,2022-May-18 03:36,36935621.621596766
1,(2016 JG),56.694720,2022-May-18 13:34,54865207.91420768
2,(2019 TZ),20.584609,2022-May-18 11:34,59997358.383933557
3,(2020 UZ),163.509359,2022-May-18 19:22,50000280.768670515
4,(2020 XG),10.128441,2022-May-18 02:52,46516703.344200312
...,...,...,...,...
61,265482 (2005 EE),249.771348,2022-May-13 23:25,72943385.193394772
62,(2016 KL),29.753812,2022-May-13 23:18,57336656.310423694
63,(2018 VZ6),28.414670,2022-May-13 19:18,24691195.493265334
64,(2021 JO2),22.158615,2022-May-13 16:45,48604582.113586124


To fit the conditions we need to change the type of values:
- take two date arguments: in "time" column to datetime
- sort by their closest approach distance: in "closest_approach_distance_km" column to numeric

In [190]:
final_planets_df["time"] = final_planets_df["time"].apply(pd.to_datetime)
final_planets_df["closest_approach_distance_m"] = final_planets_df["closest_approach_distance_m"].apply(pd.to_numeric)

The final function is:

In [191]:
def find_nearest_planets(start, end):
    result = (final_planets_df[(final_planets_df['time'] > start) & (final_planets_df["time"] < end)])\
        .sort_values(by=['closest_approach_distance_m']).reset_index()
    result = result.drop(columns="index")
    print(result)

Let's try.

In [192]:
find_nearest_planets("2022-05-17 10:00:00", "2022-05-19 02:00:00")

          name  size_estimate_m                time  \
0    (2013 UX)       171.215318 2022-05-17 13:08:00   
1    (2021 WY)        78.984635 2022-05-18 22:55:00   
2    (2021 XE)         7.644386 2022-05-17 13:29:00   
3   (2021 BF2)        24.862375 2022-05-18 15:04:00   
4   (2010 UY7)         8.581093 2022-05-18 03:36:00   
5   (2021 TA8)       248.623752 2022-05-17 18:51:00   
6   (2021 KA1)        26.154271 2022-05-18 02:01:00   
7    (2020 XG)        10.128441 2022-05-18 02:52:00   
8    (2020 UZ)       163.509359 2022-05-18 19:22:00   
9    (2016 JG)        56.694720 2022-05-18 13:34:00   
10   (2019 TZ)        20.584609 2022-05-18 11:34:00   

    closest_approach_distance_m  
0                  6.433842e+06  
1                  6.479766e+06  
2                  1.716175e+07  
3                  1.800905e+07  
4                  3.693562e+07  
5                  4.043623e+07  
6                  4.288893e+07  
7                  4.651670e+07  
8                  5.000028e+07  

# Answer: The closest approach distance might be save, let's celebrate!